In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Metropolis-Hasting algorithm

(a) Assume $\mathbf{X}=(X_1,\dots,X_{10})$ follows a multivariate standard normal distribution $\pi(\mathbf{x})$. Build a Random Walk algorithm with the Gaussian kernel to estimate $E[D]$, where $D=f(\mathbf{X})=\sqrt{\sum_{j=1}^{10}X_j^2}$. Firstly, tune the step size such that you have roughly 23.4\% acceptance rate. What step size do you choose? Then, draw 10000 samples to do the estimation, set the initial point $\mathbf{x}_0=\mathbf{0}$, set the burn-in parameter to be 1000 and set the thinning parameter to be 5. 

In [2]:
def RW(step, target, burn, x0, size, thin):
    def move(x_old):
        x_new = x_old + st.multivariate_normal(mean=np.zeros(10), cov=step ** 2).rvs()
        return x_new if st.uniform.rvs() <= (target(x_new) / target(x_old)) else x_old
    
    for b in range(burn):
        x0 = move(x0)
    
    x = np.zeros([size, 10])
    x[0] = x0
    for i in range(size - 1):
        for j in range(thin):
            x0 = move(x0)
        
        x[i + 1] = x0
    
    return x

In [3]:
np.random.seed(19971107)
target = st.multivariate_normal(mean=np.zeros(10)).pdf
X = RW(step=0.8, target=target, burn=0, x0=np.zeros(10), size=10000, thin=1)
D = np.sqrt(np.sum(X ** 2, axis=1))
print('acceptance rate:', len(set(D)) / D.size)

acceptance rate: 0.2334


In [4]:
np.random.seed(19971107)
target = st.multivariate_normal(mean=np.zeros(10)).pdf
X = RW(step=0.8, target=target, burn=1000, x0=np.zeros(10), size=10000, thin=5)
D = np.sqrt(np.sum(X ** 2, axis=1))
estimate = D.mean()
print('estimate:', estimate)

estimate: 3.086575980864723


(b) A simple techique called batching can help us to build a confidence interval with MCMC samples. We divide the above obtained 10000 samples $\{d_1,\dots,d_{10000}\}$ into 20 batches, where $d_i=f(\mathbf{x}_i)$, and $\textbf{x}_i=(x_{i1},\dots,x_{i10})$ is a sample in the MCMC sequence. Do the estimation in each batch:
$$
\overline{d}_b=\frac{1}{500}\sum_{i=500(b-1)+1}^{500b}d_i,
$$
for $b=1,\dots,20$. Estimate that
$$
s^2=\frac{1}{20(20-1)}\sum_{b=1}^{20}(\overline{d}_b-\overline{d})^2. 
$$
So, the 95\% confidence interval would be
$$
\overline{d}\pm t_{(19)}^{0.975}s. 
$$
Please give this interval. 

In [5]:
def batching(samples, B):
    averages = np.zeros(B)
    size = int(samples.size / B)
    for b in range(B):
        averages[b] = samples[size * b: size * (b + 1)].mean()
        
    average = averages.mean()
    s = np.sqrt(np.var(averages) / (B - 1))
    ts = st.t(df=B - 1).ppf(0.975) * s
    print('95% C.I.: [{}, {}]'.format(average - ts, average + ts))

In [6]:
batching(D, 20)

95% C.I.: [3.0480739092203764, 3.125078052509069]


(c) In the self-normalized IS, the optimal proposal is propotional to $\pi(\mathbf{x})|f(\mathbf{x})-E[D]|$. Replace $E[D]$ by the estimate $\hat{E}[D]$ obtained in (a), draw samples from the optimal proposal based on the Random Walk, and weight each obtained sample by the weighting function 
$$
w(\mathbf{x})=\frac{1}{|f(\mathbf{x})-\hat{E}[D]|}. 
$$
Follow a similar procedure of (a) and (b) to calculate the estimate of $E[D]$ and the 95\% confidence interval by the self-normalized IS. Does your confidence interval become wider or narrower? (hints: (1) Remember to ensure the 23.4\% acceptance rate; (2) You don't need to consider the weights of the averages for each batch in this question. )

In [7]:
np.random.seed(19971107)
target2 = lambda x: st.multivariate_normal(mean=np.zeros(10)).pdf(x) * np.abs(np.sqrt(np.sum(x ** 2)) - estimate)
X2 = RW(step=0.8, target=target2, burn=0, x0=np.zeros(10), size=10000, thin=1)
D2 = np.sqrt(np.sum(X2 ** 2, axis=1))
print('acceptance rate:', len(set(D2)) / D2.size)

acceptance rate: 0.2352


In [8]:
np.random.seed(19971107)
target = st.multivariate_normal(mean=np.zeros(10)).pdf
X2 = RW(step=0.8, target=target2, burn=1000, x0=np.zeros(10), size=10000, thin=5)
D2 = np.sqrt(np.sum(X ** 2, axis=1))
W = 1 / np.abs(D2 - estimate)
estimate2 = np.sum(W * D2) / np.sum(W)
print('estimate:', estimate2)

estimate: 3.083841117561734


In [9]:
def batching(samples, weights, B):
    averages = np.zeros(B)
    size = int(samples.size / B)
    for b in range(B):
        w = weights[size * b: size * (b + 1)]
        averages[b] = np.sum(w * samples[size * b: size * (b + 1)]) / np.sum(w)
        
    average = averages.mean()
    s = np.sqrt(np.var(averages) / (B - 1))
    ts = st.t(df=B - 1).ppf(0.975) * s
    print('95% C.I.: [{}, {}]'.format(average - ts, average + ts))

In [10]:
batching(D2, W, 20)

95% C.I.: [3.077353218409959, 3.0894742759791107]
